## objectives of pre-processing
    
- data should be 2D array
- input data has to be numbers
- no nan or inf
- coloum are scaled to similar ranges (mean=0, variance=1)
- coloums should not be coliner (cx1!=k*cx2)
- rows should not be causally dependent
- data should be 100 times larger then the number of coloums 

## install python libraries to local environment

In [1]:
# install required packages
# /apps/pangeo/2020.05/bin/python -m pip install --user rasterio
# /apps/pangeo/2020.05/bin/python -m pip install --user wheel
# /apps/pangeo/2020.05/bin/python -m pip install --user fiona
# /apps/pangeo/2020.05/bin/python -m pip install --user xgboost
# /apps/pangeo/2020.05/bin/python -m pip install --user lightgbm
# /apps/pangeo/2020.05/bin/python -m pip install --user graphvizfloat32

## import python libraries 

In [ ]:
import os
import pathlib
from pprint import pprint
from pathlib import Path
import csv

In [ ]:
import ray
import rasterio
import fiona
import pandas as pd
import xgboost as xgb
import numpy as np


from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
    

In [4]:
%load_ext memory_profiler

## define input datasets

- features 
- targets 
- out of sample 
- area of interest

In [48]:
%%time
%%memit
feature_file_paths = [
Path("/g/data/ge3/sheece/LOC_distance_to_coast.tif"),
Path("/g/data/ge3/sheece/mrvbf_9.tif"),
Path("/g/data/ge3/sheece/relief_mrvbf_3s_mosaic.tif"),
Path("/g/data/ge3/sheece/relief_elev_focalrange1000m_3s.tif"),
Path("/g/data/ge3/sheece/relief_elev_focalrange300m_3s.tif"),
Path("/g/data/ge3/sheece/saga_wetSM_85_resampled.tif"),
Path("/g/data/ge3/sheece/tpi_300.tif"),
Path("/g/data/ge3/sheece/slope_fill2.tif"),
Path("/g/data/ge3/sheece/dem_fill.tif"),
Path("/g/data/ge3/sheece/3dem_mag2.tif"),
Path("/g/data/ge3/sheece/3dem_mag1_fin.tif"),
Path("/g/data/ge3/sheece/3dem_mag0.fin.tif"),
Path("/g/data/ge3/sheece/relief_roughness.tif"),
Path("/g/data/ge3/sheece/LATITUDE_GRID1_clip.tif"),
Path("/g/data/ge3/sheece/LATITUDE_GRID1_clip.tif"),
Path("/g/data/ge3/sheece/Dose_2016.tif"),
Path("/g/data/ge3/sheece/Potassium_2016.tif"),
Path("/g/data/ge3/sheece/Thorium_2016.tif"),
Path("/g/data/ge3/sheece/Rad2016U_Th.tif"),
Path("/g/data/ge3/sheece/Rad2016K_Th.tif"),
Path("/g/data/ge3/sheece/national_Wii_RF_multirandomforest_prediction.tif"),
Path("/g/data/ge3/sheece/si_geol1.tif"),
Path("/g/data/ge3/sheece/ceno_euc_aust1.tif"),
Path("/g/data/ge3/sheece/Grav_lane_clip.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-ND-NIR-GREEN.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-ND-RED-BLUE.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-ND-SWIR1-SWIR2.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg_BLUE+SWIR2.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-ND-SWIR1-NIR.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-CLAY-PC2.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-ND-RED-BLUE.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-RED.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-GREEN.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-l8-all-85m-avg-BLUE.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-l8-all-85m-avg-NIR.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-SWIR1.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/be-30y-85m-avg-SWIR2.filled.lzw.nodata.tif"),
Path("/g/data/ge3/sheece/s2-dpca-85m.tif"),
Path("/g/data/ge3/sheece/water-85m.tif"),
Path("/g/data/ge3/sheece/clim_EPA_albers.tif"),
Path("/g/data/ge3/sheece/Clim_Prescott_LindaGregory.tif"),
Path("/g/data/ge3/sheece/clim_PTA_albers.tif"),
Path("/g/data/ge3/sheece/clim_WDA_albers.tif"),
Path("/g/data/ge3/sheece/clim_RSM_albers.tif")
]

peak memory: 190.86 MiB, increment: 0.01 MiB
CPU times: user 93.2 ms, sys: 46.6 ms, total: 140 ms
Wall time: 291 ms


In [41]:
# #target dataset complete
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled.cpg 
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled.dbf
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled.prj
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled.shp
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled.shx

# #target dataset small
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled_small.cpg
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled_small.dbf
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled_small.prj
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled_small.sbn
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled_small.sbx
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled_small.shp
# /g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled_small.shx
target_file_path = Path("/g/data/ge3/sheece/0_50cm_2021_albers_C_sm_T_resampled.shp")

In [42]:
# define a shape for area of intrest
area_of_interest_file_path = None

In [43]:
# #OOS
# /g/data/ge3/sheece/0_50cm_2021_albers_C_oos.cpg
# /g/data/ge3/sheece/0_50cm_2021_albers_C_oos.dbf
# /g/data/ge3/sheece/0_50cm_2021_albers_C_oos.prj
# /g/data/ge3/sheece/0_50cm_2021_albers_C_oos.sbn
# /g/data/ge3/sheece/0_50cm_2021_albers_C_oos.sbx
# /g/data/ge3/sheece/0_50cm_2021_albers_C_oos.shp
# /g/data/ge3/sheece/0_50cm_2021_albers_C_oos.shp.xml
# /g/data/ge3/sheece/0_50cm_2021_albers_C_oos.shx

out_of_sample_file_path = Path("/g/data/ge3/sheece/0_50cm_2021_albers_C_oos.shp")

## Standardising Datasets

In [44]:
root = Path('/g/data/ge3/sheece')
if not os.path.isdir(root/"old"):
    os.mkdir(root/"old") 

In [31]:
import shutil
from rasterio.warp import calculate_default_transform, reproject, Resampling

# check projection 
crs_epsg3577 = rasterio.crs.CRS.from_string('EPSG:3577')

for feature_file_path in feature_file_paths:
    with rasterio.open(feature_file_path) as src:
        if crs_epsg3577 != src.crs:
                print("Converting dataset: "+str(feature_file_path))
                transform, width, height = calculate_default_transform(
                    src.crs, 
                    crs_epsg3577, 
                    src.width, 
                    src.height, 
                    *src.bounds)
                kwargs = src.meta.copy()
                kwargs.update({'crs': crs_epsg3577,'transform': transform, 'width': width,'height': height})
                
                
                with rasterio.open(root/ (feature_file_path.stem+'_resampled.tif'), 'w', **kwargs) as dst:
                    reproject(
                        source=rasterio.band(src, 1),
                        destination=rasterio.band(dst, 1),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=crs_epsg3577,
                        resampling=Resampling.nearest
                    )
                    print(dst.meta)
                    
                shutil.move(feature_file_path, root/("old/"+feature_file_path.name))
                print(root/ (feature_file_path.stem+'_resampled.tif')


Converting dataset: /g/data/ge3/sheece/saga_wetSM_85.tif
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -99999.0, 'width': 37085, 'height': 18982, 'count': 1, 'crs': CRS.from_epsg(3577), 'transform': Affine(84.99895098729797, 0.0, -1555680.0000045914,
       0.0, -84.99895098729797, -1052891.999967362)}


In [46]:
# apply crop on target and feature datasets according to area of intrest 
import fiona
import rasterio
from rasterio.mask import mask

if area_of_interest_file_path is not None:
    with fiona.open(area_of_interest_file_path) as shapefile:
        geoms = [feature["geometry"] for feature in shapefile]

    for feature_file_path in feature_file_paths:
        if "cropped" not in feature_file_path.name:
            print("Cropping : "+feature_file_path.stem)
            # load the raster, mask it by the polygon and crop it
            with rasterio.open(feature_file_path) as src:
                out_image, out_transform = mask(src, geoms, crop=True)
            out_meta = src.meta.copy()

            # save the resulting raster  
            out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
            })   

            with rasterio.open(root/(feature_file_path.stem+'_cropped.tif'), "w", **out_meta) as dest:
                dest.write(out_image)

            shutil.move(feature_file_path, root/("old/"+feature_file_path.name))

In [47]:
# apply crop vector files(.shp)
import subprocess

if area_of_interest_file_path is not None:
    if "cropped" not in target_file_path.name:
        print("Cropping: "+target_file_path.name)
        clipped_file = root/(target_file_path.stem+'_cropped.shp')
        callstr = ['ogr2ogr',
                   '-clipsrc',
                   area_of_interest_file_path,
                   clipped_file,
                   target_file_path] 
        proc = subprocess.Popen(callstr, stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        stdout,stderr=proc.communicate()

        shutil.move(target_file_path, root/("old/"+target_file_path.name))

In [84]:
# create iterators ( C++ pointers) to inputs feature dataset and target dataset
datasets = []
input_feature_files = feature_file_paths
for feature_file_path in feature_file_paths: 
    datasets.append(rasterio.open(feature_file_path))

target_handle = fiona.open(target_file_path)

In [82]:
pprint(target_handle.meta)

{'crs': {'init': 'epsg:3577'},
 'crs_wkt': 'PROJCS["GDA94 / Australian '
            'Albers",GEOGCS["GDA94",DATUM["Geocentric_Datum_of_Australia_1994",SPHEROID["GRS '
            '1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6283"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4283"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",-18],PARAMETER["standard_parallel_2",-36],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",132],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3577"]]',
 'driver': 'ESRI Shapefile',
 'schema': {'geometry': 'Point',
            'properties': OrderedDict([('con', 'float:24.15')])}}


In [98]:
%%time
%%memit

head_row = ['target']
for input_feature_file in input_feature_files:
    head_row.append(input_feature_file)

csv_rowlist = [head_row]
with open(root/'input_dataset.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for target in target_handle:
        value = target["properties"]["con"]
        x, y = target["geometry"]["coordinates"]
        new_row = [value]
        
        for dataset in datasets:
            new_row.append(next(dataset.sample([(x, y)]))[0])
        csv_rowlist.append(new_row)

        if len(csv_rowlist)%1000==0:
            writer.writerows(csv_rowlist)
            csv_rowlist = []

peak memory: 12738.79 MiB, increment: 2586.16 MiB
CPU times: user 20min 17s, sys: 2min 15s, total: 22min 32s
Wall time: 28min 27s


In [99]:
target_handle.close()
for dataset in datasets: 
    dataset.close()

In [20]:
if not os.path.isdir(root/"models"):
    os.mkdir(root/"models") 

In [ ]:
### exploring dataset csv

In [100]:
import dask.dataframe as dd

filename = root/'input_dataset.csv'
df = dd.read_csv(filename, dtype='str')

In [101]:
df.head(2)

,target,/g/data/ge3/sheece/LOC_distance_to_coast.tif,/g/data/ge3/sheece/mrvbf_9.tif,/g/data/ge3/sheece/relief_mrvbf_3s_mosaic.tif,/g/data/ge3/sheece/relief_elev_focalrange1000m_3s.tif,/g/data/ge3/sheece/relief_elev_focalrange300m_3s.tif,/g/data/ge3/sheece/saga_wetSM_85_resampled.tif,/g/data/ge3/sheece/tpi_300.tif,/g/data/ge3/sheece/slope_fill2.tif,/g/data/ge3/sheece/dem_fill.tif,...,/g/data/ge3/sheece/be-l8-all-85m-avg-NIR.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-SWIR1.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-SWIR2.filled.lzw.nodata.tif,/g/data/ge3/sheece/s2-dpca-85m.tif,/g/data/ge3/sheece/water-85m.tif,/g/data/ge3/sheece/clim_EPA_albers.tif,/g/data/ge3/sheece/Clim_Prescott_LindaGregory.tif,/g/data/ge3/sheece/clim_PTA_albers.tif,/g/data/ge3/sheece/clim_WDA_albers.tif,/g/data/ge3/sheece/clim_RSM_albers.tif
0,0.012695,2.0063581,1,4,21.217691,9.958379,10.004804,-0.26448622,0.94074196,205.45955,...,0.2428,0.2789,0.1753,192,0.026450625,1833.6868,0.4862007,803.89575,-1029.791,19.99033
1,0.004673,2.0257869,1,0,16.920124,8.870855,9.931655,0.07641165,1.034455,213.70831,...,0.2298,0.2535,0.1592,210,0.017286092,1826.2247,0.4858998,802.7006,-1023.5242,19.941477


In [102]:
# create a new dataframe with only 'target=0.007172' service calls
target_df=df[df.target=='0.012695']
target_df.target.count()
target_df.compute()

,target,/g/data/ge3/sheece/LOC_distance_to_coast.tif,/g/data/ge3/sheece/mrvbf_9.tif,/g/data/ge3/sheece/relief_mrvbf_3s_mosaic.tif,/g/data/ge3/sheece/relief_elev_focalrange1000m_3s.tif,/g/data/ge3/sheece/relief_elev_focalrange300m_3s.tif,/g/data/ge3/sheece/saga_wetSM_85_resampled.tif,/g/data/ge3/sheece/tpi_300.tif,/g/data/ge3/sheece/slope_fill2.tif,/g/data/ge3/sheece/dem_fill.tif,...,/g/data/ge3/sheece/be-l8-all-85m-avg-NIR.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-SWIR1.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-SWIR2.filled.lzw.nodata.tif,/g/data/ge3/sheece/s2-dpca-85m.tif,/g/data/ge3/sheece/water-85m.tif,/g/data/ge3/sheece/clim_EPA_albers.tif,/g/data/ge3/sheece/Clim_Prescott_LindaGregory.tif,/g/data/ge3/sheece/clim_PTA_albers.tif,/g/data/ge3/sheece/clim_WDA_albers.tif,/g/data/ge3/sheece/clim_RSM_albers.tif
0,0.012695,2.0063581,1,4,21.217691,9.958379,10.004804,-0.26448622,0.94074196,205.45955,...,0.2428,0.2789,0.1753,192,0.026450625,1833.6868,0.4862007,803.89575,-1029.791,19.99033
35843,0.012695,2.601555,5,5,15.137862,3.4422703,10.8057785,-0.17277434,0.27034372,344.0473,...,0.2552,0.3298,0.2618,127,0.026767487,1789.1764,0.12620586,366.281,-1422.8954,22.019802
114269,0.012695,1.8316871,4,0,49.2953,16.808044,9.389273,0.9480591,1.3146253,503.0447,...,0.1809,0.2831,0.2236,211,0.0041432963,2133.3025,0.16949372,364.40176,-1768.9009,23.08512


In [ ]:
#!pangeo.end.sh

In [96]:
!/apps/gdal/3.0.2/bin/gdalinfo /g/data/ge3/sheece/water-85m.tif

Driver: GTiff/GeoTIFF
Files: /g/data/ge3/sheece/water-85m.tif
Size is 37085, 18982
Coordinate System is:
BOUNDCRS[
    SOURCECRS[
        PROJCRS["GDA94 / Australian Albers",
            BASEGEOGCRS["GDA94",
                DATUM["Geocentric Datum of Australia 1994",
                    ELLIPSOID["GRS 1980",6378137,298.257222101,
                        LENGTHUNIT["metre",1]]],
                PRIMEM["Greenwich",0,
                    ANGLEUNIT["degree",0.0174532925199433]],
                ID["EPSG",4283]],
            CONVERSION["Australian Albers",
                METHOD["Albers Equal Area",
                    ID["EPSG",9822]],
                PARAMETER["Latitude of false origin",0,
                    ANGLEUNIT["degree",0.0174532925199433],
                    ID["EPSG",8821]],
                PARAMETER["Longitude of false origin",132,
                    ANGLEUNIT["degree",0.0174532925199433],
                    ID["EPSG",8822]],
                PARAMETER["Latitude of 1st standa

In [ ]:
#  NoData Value=-9999
# replace with mean, or drop the row
# mean of the points around it !!!!
imputation mean --- 9 values....

pixels around it!!


In [103]:
train = np.genfromtxt(str(root)+'/input_dataset.csv', delimiter=',')
train = train.astype(np.float32)
train = train[~np.isnan(train).any(axis=1)]
train = train[~(train == -9999.0).any(axis=1)]

In [115]:
import pandas as pd 
head_row = ['target']
for input_feature_file in input_feature_files:
    head_row.append(input_feature_file)
    
pd.DataFrame(train).to_csv(str(root)+'/input_dataset_filtered.csv',header=head_row, index=None)

In [116]:
import dask.dataframe as dd

filename =str(root)+'/input_dataset_filtered.csv'
df = dd.read_csv(filename, dtype='str')

In [117]:
df.head(2)

,target,/g/data/ge3/sheece/LOC_distance_to_coast.tif,/g/data/ge3/sheece/mrvbf_9.tif,/g/data/ge3/sheece/relief_mrvbf_3s_mosaic.tif,/g/data/ge3/sheece/relief_elev_focalrange1000m_3s.tif,/g/data/ge3/sheece/relief_elev_focalrange300m_3s.tif,/g/data/ge3/sheece/saga_wetSM_85_resampled.tif,/g/data/ge3/sheece/tpi_300.tif,/g/data/ge3/sheece/slope_fill2.tif,/g/data/ge3/sheece/dem_fill.tif,...,/g/data/ge3/sheece/be-l8-all-85m-avg-NIR.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-SWIR1.filled.lzw.nodata.tif,/g/data/ge3/sheece/be-30y-85m-avg-SWIR2.filled.lzw.nodata.tif,/g/data/ge3/sheece/s2-dpca-85m.tif,/g/data/ge3/sheece/water-85m.tif,/g/data/ge3/sheece/clim_EPA_albers.tif,/g/data/ge3/sheece/Clim_Prescott_LindaGregory.tif,/g/data/ge3/sheece/clim_PTA_albers.tif,/g/data/ge3/sheece/clim_WDA_albers.tif,/g/data/ge3/sheece/clim_RSM_albers.tif
0,0.012695,2.0063581,1.0,4.0,21.217691,9.958379,10.004804,-0.26448622,0.94074196,205.45955,...,0.2428,0.2789,0.1753,192.0,0.026450625,1833.6868,0.4862007,803.89575,-1029.791,19.99033
1,0.004673,2.0257869,1.0,0.0,16.920124,8.870855,9.931655,0.07641165,1.034455,213.70831,...,0.2298,0.2535,0.1592,210.0,0.017286092,1826.2247,0.4858998,802.7006,-1023.5242,19.941477
